In [1]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.nn import NNConv

from sklearn.mixture import GaussianMixture 

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['date'] = pd.to_datetime(data.date)
    print('Days: ',len(set(data.date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = '20190402_TaipeiEdgesDatewise.csv'
events_data =dataDir+'TaipeiEvents.csv'

In [5]:
file = dataDir + dataFile
dataRaw = loadData(file)

Raw shape:  (7374816, 5)
Days:  638


In [6]:
# dataIn =  pd.pivot_table(dataRaw, values='amount', index=['date'],
#                     columns=['end_id'], aggfunc=np.sum, fill_value=0)
# dataIn.head(1)

In [7]:
dataRaw.head(2)

,Unnamed: 0,start_id,end_id,date,amount
0,0,BL01,BL01,2017-01-01,70.0
1,1,BL01,BL01,2017-01-02,40.0


In [8]:
# dataOut =  pd.pivot_table(dataRaw, values='amount', index=['date'],
#                     columns=['start_id'], aggfunc=np.sum, fill_value=0)
# dataOut.head(1)

In [9]:
# matrix = np.stack((dataOut.values, dataIn.values),-1)
# matrix.shape

In [10]:
dataEdge = getTimeSeries(dataRaw)
matrix = dataEdge.values
matrix = matrix.astype(float)
matrix.shape

(638, 11664)

In [11]:
# for i in range(matrix.shape[1]):
#         matrix[:, i] = (matrix[:, i] - np.mean(matrix[:, i])) / (np.std(matrix[:, i]))

In [12]:
for i in range(matrix.shape[0]):
        matrix[i,:] = (matrix[i,:] - np.mean(matrix[i,:])) / (np.std(matrix[i,:]))

In [13]:
stations = list(set(dataRaw.start_id))
n= len(stations)
edge_index = [[a//n,a%n] for a in range(n*n)]
edge_index = torch.tensor(edge_index, dtype=torch.long)

In [14]:
dates = list(dataEdge.index)
DOW = list(pd.to_datetime(dataEdge.index.values).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [15]:
dataList = []

for i in range(len(DOW)):
    x = torch.tensor(np.ones((n,1)), dtype=torch.float)
    y = torch.tensor(np.array([DOW[i]]), dtype=torch.long)
    e = torch.tensor(matrix[i].reshape(matrix.shape[1],1), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.t().contiguous(),y=y,edge_attr=e)
    dataList.append(data)

dataList[0]

Data(edge_attr=[11664, 1], edge_index=[2, 11664], x=[108, 1], y=[1])

In [16]:
n_classes = 2
featureDim = 1
hiddenDim = [5,100,20]
num_edge_features = 1

In [17]:
# NNConv?

In [18]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.conv1 = GCNConv(featureDim, hiddenDim[0])
        self.conv1Lin = nn.Linear(1, featureDim * hiddenDim[0])
        self.conv1 = NNConv(featureDim, hiddenDim[0],self.conv1Lin)
        #self.conv1 = NNConv(featureDim, hiddenDim[0],nn.Linear(1, 1000))
        
        #self.conv2 = GCNConv(hiddenDim[0], hiddenDim[1])
        #self.conv2Lin = nn.Linear(1, hiddenDim[0]*hiddenDim[1])
        #self.conv2 = NNConv(hiddenDim[0], hiddenDim[1],self.conv2Lin)
        
        self.linear1 = nn.Linear(n*hiddenDim[0], hiddenDim[1])
        self.linear2 = nn.Linear(hiddenDim[1], hiddenDim[2])
        self.linear3 = nn.Linear(hiddenDim[2], n_classes)

        self.init_weights(self.linear1)
        self.init_weights(self.linear2)
        self.init_weights(self.linear3)
        self.init_weights(self.conv1Lin)
        #self.init_weights(self.conv2Lin)
        
    def forward(self, data):
        x, edge_index,edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index,edge_attr)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        #x = self.conv2(x, edge_index,edge_attr)
        #x = F.relu(x)
        
        x = x.view(-1,n*hiddenDim[0])
        x = self.linear1(x)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return F.log_softmax(x, dim=1)
        
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x, edge_index,edge_attr = data.x, data.edge_index,data.edge_attr
        x = self.conv1(x, edge_index,edge_attr)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        #x = self.conv2(x, edge_index,edge_attr)
        #x = F.relu(x)
        x = x.view(-1,n*hiddenDim[0])
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x
            

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [20]:
device

device(type='cuda')

In [21]:
#optimizer = torch.optim.Adam(model.parameters(), lr=0.000001, weight_decay=5e-4)
num_epochs = 100

In [22]:
model.train()

for epoch in range(num_epochs):
    epochLoss = 0
    for data in dataList:
        #data.view(1,-1,-1)
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
#         print(out.shape)
#         print(data.y.shape)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        
        epochLoss += loss.item()
    
#     if epoch == 500:
#         optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=5e-4)
        
    print('epoch [{}/{}], loss:{:.4f}'
    .format(epoch + 1, num_epochs, np.mean(epochLoss)))

epoch [1/100], loss:625.6123
epoch [2/100], loss:172.8031
epoch [3/100], loss:152.6079
epoch [4/100], loss:82.6175
epoch [5/100], loss:90.8204
epoch [6/100], loss:102.8731
epoch [7/100], loss:57.3253
epoch [8/100], loss:46.8371
epoch [9/100], loss:78.9268
epoch [10/100], loss:180.2650
epoch [11/100], loss:85.0397
epoch [12/100], loss:62.0268
epoch [13/100], loss:26.4913
epoch [14/100], loss:40.5989
epoch [15/100], loss:27.5792
epoch [16/100], loss:88.1200
epoch [17/100], loss:128.7302
epoch [18/100], loss:33.2342
epoch [19/100], loss:73.9957
epoch [20/100], loss:77.1683
epoch [21/100], loss:39.4795
epoch [22/100], loss:31.6922
epoch [23/100], loss:38.3881
epoch [24/100], loss:18.6366
epoch [25/100], loss:26.1933
epoch [26/100], loss:61.0188
epoch [27/100], loss:19.7734
epoch [28/100], loss:17.6079
epoch [29/100], loss:18.0494
epoch [30/100], loss:15.1032
epoch [31/100], loss:28.2577
epoch [32/100], loss:43.6651
epoch [33/100], loss:53.4929
epoch [34/100], loss:8.5669
epoch [35/100], lo

In [23]:
model.eval()
with torch.no_grad():
    y = DOW
    predList = []
    for data in dataList:
        data = data.to(device)
        pred = model(data).cpu().numpy()
        predList.append(np.argmax(pred))
    
    print('Accuracy: {:.4f}'.format( np.sum(np.array(predList) == np.array(y)) / len(predList) ))

Accuracy: 0.9906


In [24]:
representation = np.empty((matrix.shape[0],hiddenDim[2]))
#representationR = np.empty((matrix.shape[0],hiddenDim[2]))
prediction = np.empty((matrix.shape[0],n_classes))
model.eval()
with torch.no_grad():
    for i,data in enumerate(dataList):
        data = data.to(device)
        representation[i] = model.representation(data).cpu().numpy()
        #representationR[i] = model.representationR(data).cpu().numpy()
        prediction[i] = model(data).cpu().numpy()
        predList.append(np.argmax(pred))

In [25]:
representation.shape

(638, 20)

In [26]:
prediction.shape

(638, 2)

In [27]:
# import events data
events_data =dataDir+'TaipeiEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [28]:
df_events.head()

,Type,Name,Date
0,National holiday,Republic Day/New Year's Day observed,2017-01-02
1,National holiday,Chinese New Year's Eve,2017-01-27
2,National holiday,Chinese New Year's Day,2017-01-28
3,National holiday,Chinese New Year Holiday 1,2017-01-29
4,National holiday,Chinese New Year Holiday 2,2017-01-30


In [29]:
holidayDates = df_events[df_events.Type == 'National holiday'].Date

In [30]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = [str(d.date()) for d in dates]

In [31]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [32]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [33]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [34]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [35]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
Iteration 1
Iteration 2
Itera

In [36]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.285714,0.001642,0.833333,0.172414,0.01
1,Global,0.375000,0.016420,0.473684,0.310345,0.03
2,Global,0.444444,0.021346,0.480000,0.413793,0.04
3,Global,0.533333,0.024631,0.516129,0.551724,0.05
4,Global,0.567164,0.031199,0.500000,0.655172,0.06
5,Global,0.602740,0.036125,0.500000,0.758621,0.07
6,Global,0.650000,0.041051,0.509804,0.896552,0.08
7,Global,0.565217,0.060755,0.412698,0.896552,0.10
8,Global,0.358974,0.162562,0.220472,0.965517,0.20
9,Global,0.254545,0.267652,0.146597,0.965517,0.30
